In [1]:
import numpy as np
import pandas as pd
import random
from os import listdir

In [2]:
# 取出5种属性的所有属性值
def find_total_attrs(path):
	attrs = listdir(path)
	total_attrs = []
	for attr in attrs:
		vals = listdir(path + attr)
		vals= [ x.split('=')[1].split('.')[0] for x in vals ]
		if 'unknown' in vals:
			vals.remove('unknown')
		total_attrs.append(vals)
	# print(total_attrs)
	return total_attrs


# 一共5层，每层1~5种root cause,共25种
def find_root_causes(total_attrs):
	layers, kinds = 5, 5
	root_causes = []
	for i in range(layers):
		layer_vals = []
		for j in range(kinds):
			vals = []
			# 从5个属性里随机选择 i 个(第一层1个，第二层2个...)
			random_nums = random.sample([x for x in range(5)], i + 1)	
			for k in range(i + 1):
				vals.append( random.sample(total_attrs[random_nums[k]], 1)[0] )	
			layer_vals.append(vals)
			temp = layer_vals[:]		# 深拷贝
			root_causes.append(temp)
	return root_causes


## 随机找出25种根因

In [8]:
total_attrs = find_total_attrs('../data/result/')
root_causes = find_root_causes(total_attrs)
np.save('../data/root_causes.npy', root_causes)

for root_cause in root_causes:
	print(root_cause)

[['l2']]
[['l2'], ['e01']]
[['l2'], ['e01'], ['e06']]
[['l2'], ['e01'], ['e06'], ['e07']]
[['l2'], ['e01'], ['e06'], ['e07'], ['l3']]
[['l4', 'c8']]
[['l4', 'c8'], ['p26', 'c1']]
[['l4', 'c8'], ['p26', 'c1'], ['c8', 'l2']]
[['l4', 'c8'], ['p26', 'c1'], ['c8', 'l2'], ['p12', 'i147']]
[['l4', 'c8'], ['p26', 'c1'], ['c8', 'l2'], ['p12', 'i147'], ['i61', 'l3']]
[['i123', 'l1', 'c6']]
[['i123', 'l1', 'c6'], ['p02', 'l2', 'e02']]
[['i123', 'l1', 'c6'], ['p02', 'l2', 'e02'], ['p14', 'c6', 'i106']]
[['i123', 'l1', 'c6'], ['p02', 'l2', 'e02'], ['p14', 'c6', 'i106'], ['c6', 'p20', 'i141']]
[['i123', 'l1', 'c6'], ['p02', 'l2', 'e02'], ['p14', 'c6', 'i106'], ['c6', 'p20', 'i141'], ['i118', 'c1', 'l2']]
[['p32', 'l2', 'e12', 'c6']]
[['p32', 'l2', 'e12', 'c6'], ['e01', 'p08', 'c1', 'l4']]
[['p32', 'l2', 'e12', 'c6'], ['e01', 'p08', 'c1', 'l4'], ['c7', 'e09', 'p32', 'l3']]
[['p32', 'l2', 'e12', 'c6'], ['e01', 'p08', 'c1', 'l4'], ['c7', 'e09', 'p32', 'l3'], ['c4', 'l4', 'p29', 'e05']]
[['p32', 'l2', '

## 随机找出25个timestamp注入异常

In [11]:
# 计算异常
def compute_anomaly(path):
    o = open(path, 'rb')
    df = pd.read_csv(o)
    length = len(df)

    # 随机选取一个时间戳(为方便计算不选头尾处时间戳)
    rand_idx = random.randint(150, length - 150)
    timestamp = df.loc[rand_idx][0]						# 随机选择的timestamp
    temp_df = df[rand_idx - 144 : rand_idx + 144]		# 时间间隔为5分钟，共14天数据，选取一天的数据
    kpi = temp_df['KPI'].as_matrix()
    kpi = [float(i) for i in kpi]
    # print(kpi)
    mean = np.mean(kpi)
    std = np.std(kpi)
    total_anomaly = int(mean + 5*std)					# 此timestamp上的总KPI异常
    # print(total_anomaly)
    return timestamp, total_anomaly


# 分发异常
def distribute_anomaly(root_cause, path):
    timestamp, total_anomaly = compute_anomaly(path)
    print(root_cause, timestamp)
    path2 = '../data/origin/' + str(timestamp) + '.csv'
    path3 = '../data/anomaly/' + str(timestamp) + '.csv'
    locs = []           # 后代叶子元素的索引
    o = open(path2, 'rb')
    df = pd.read_csv(o, names  = ['i', 'e', 'c', 'p', 'l', 'kpi'])
    for x in root_cause:
        for idx in df.index:
            i, e, c, p, l = df.loc[idx]['i'], df.loc[idx]['e'], df.loc[idx]['c'], df.loc[idx]['p'], df.loc[idx]['l']
            if i in x or e in x or c in x or p in x or l in x:
                locs.append(idx)

    # 若此时间戳不含此属性值，则重新找一个时间戳
    if len(locs) == 0:
        distribute_anomaly(root_cause, path)
        return

    print(locs)
    anomaly = int(total_anomaly / len(locs))
    df_out = pd.DataFrame(columns = ['i', 'e', 'c', 'p', 'l', 'kpi'])
    for idx in locs:
        i, e, c, p, l = df.loc[idx]['i'], df.loc[idx]['e'], df.loc[idx]['c'], df.loc[idx]['p'], df.loc[idx]['l']
#                 print(i, e, c, p, l)
        df_out.loc[0] = {'i':i, 'e':e, 'c':c, 'p':p, 'l':l, 'kpi':df.loc[idx]['kpi'] + anomaly}
        df = pd.concat([df[:idx], df_out, df[idx+1:]], ignore_index=True)
    csvfile = open(path3, 'w')
    df.to_csv(path3, header=None, index=None)

In [13]:
# 使用25种根因随机选择timestamp注入异常
path = '../data/KPI_new.csv'
for root_cause in root_causes:
    distribute_anomaly(root_cause, path)

[['l2']] 1536895800000
[2, 8, 11, 15, 23, 25, 31, 38, 59, 95, 101, 119, 120, 121, 153, 182, 184, 185, 186, 188, 195, 242, 258, 259, 270, 271, 273, 278, 296, 297, 302, 324, 336, 342, 349, 355, 358, 387, 389, 410, 415, 421, 426, 432, 434, 436, 445, 452, 475, 476, 509, 511, 512, 521, 522, 527, 532, 541, 542, 545, 553, 571, 579, 582, 591, 592, 594, 597, 608, 624, 626, 642, 645, 646, 661, 666, 667, 676, 681, 716, 733, 747, 748, 750, 751, 752, 753, 754, 756, 761, 762, 769, 793, 797, 800, 805, 817, 827, 835, 883, 887, 901, 910, 912, 914, 915, 919, 922, 923, 929, 954, 960, 964, 970, 971, 972, 979, 991, 1003, 1049, 1055, 1056, 1062, 1064, 1068, 1069, 1081, 1088, 1092, 1093, 1094, 1095, 1108, 1115, 1116, 1126, 1127, 1150, 1162, 1164, 1169, 1171, 1179, 1180, 1184, 1187, 1197, 1206, 1232, 1242, 1252, 1259, 1274, 1281, 1288, 1295, 1296, 1297, 1298, 1304, 1317, 1325, 1341, 1345, 1348, 1362, 1366, 1387, 1393, 1399, 1409, 1411, 1415, 1431, 1435, 1440, 1468, 1472, 1473, 1474, 1480, 1488, 1502, 1503, 15

[['l2'], ['e01'], ['e06'], ['e07'], ['l3']] 1536223800000
[6, 9, 10, 18, 20, 72, 75, 79, 81, 84, 89, 91, 98, 106, 112, 129, 130, 131, 134, 137, 141, 149, 167, 169, 176, 186, 188, 192, 199, 210, 218, 222, 265, 269, 271, 277, 287, 289, 297, 322, 326, 333, 346, 365, 381, 390, 391, 406, 409, 410, 413, 424, 440, 446, 457, 458, 464, 465, 469, 473, 474, 490, 503, 508, 536, 537, 544, 545, 546, 549, 575, 596, 602, 604, 607, 609, 616, 634, 644, 655, 664, 667, 668, 675, 688, 692, 697, 698, 702, 705, 723, 737, 739, 751, 753, 759, 762, 770, 783, 786, 799, 801, 822, 835, 837, 838, 847, 850, 861, 862, 864, 900, 903, 904, 906, 911, 912, 913, 920, 921, 922, 927, 933, 948, 972, 984, 987, 997, 998, 1000, 1001, 1018, 1025, 1027, 1028, 1029, 1032, 1057, 1064, 1065, 1081, 1082, 1084, 1086, 1109, 1120, 1128, 1136, 1140, 1145, 1146, 1158, 1165, 1173, 1213, 1215, 1220, 1221, 1222, 1228, 1235, 1241, 1250, 1255, 1257, 1263, 1290, 1296, 1297, 1302, 1304, 1306, 1310, 1313, 1314, 1321, 1332, 1333, 1356, 1366, 1375,

[['l4', 'c8']] 1536479100000
[4, 67, 68, 138, 168, 229, 234, 273, 309, 314, 329, 378, 458, 459, 473, 532, 533, 624, 653, 672, 679, 697, 717, 751, 759, 815, 837, 854, 889, 895, 896, 927, 970, 974, 1037, 1039, 1042, 1048, 1081, 1089, 1118, 1186, 1196, 1197, 1212, 1265, 1269, 1276, 1283, 1285, 1294, 1383, 1396, 1431, 1433, 1449, 1453, 1493, 1518, 1547, 1590, 1592, 1596, 1605, 1608, 1618, 1633, 1643, 1659, 1660, 1661, 1685, 1741, 1743, 1773, 1779, 1820, 1827, 1854, 1888, 1923, 1931, 1956, 1958, 1983, 1997, 2020, 2021, 2094, 2095, 2102, 2118, 2124, 2152, 2179, 2186, 2205, 2230, 2245, 2258, 2262, 2275, 2306, 2316, 2323, 2380, 2381, 2383, 2384, 2404, 2452, 2525, 2541, 2565, 2568, 2595, 2597, 2624, 2642, 2663, 2664, 2665, 2748, 2784, 2813, 2818, 2836, 2848, 2885, 2895, 2897, 2900]
[['l4', 'c8'], ['p26', 'c1']] 1536637800000
[8, 10, 29, 57, 112, 119, 120, 121, 161, 163, 170, 173, 230, 234, 235, 239, 278, 284, 311, 321, 324, 389, 403, 414, 461, 477, 479, 480, 482, 492, 525, 552, 556, 606, 628, 6

[['l4', 'c8'], ['p26', 'c1'], ['c8', 'l2']] 1536241800000
[5, 15, 19, 66, 96, 137, 162, 170, 213, 217, 218, 219, 235, 277, 296, 348, 349, 350, 409, 506, 560, 561, 563, 578, 586, 590, 609, 640, 645, 646, 648, 650, 652, 724, 725, 726, 727, 830, 831, 878, 939, 940, 941, 942, 1008, 1033, 1065, 1067, 1075, 1148, 1154, 1202, 1210, 1225, 1239, 1265, 1283, 1284, 1316, 1339, 1340, 1343, 1369, 1411, 1414, 1415, 1416, 1468, 1513, 1579, 1600, 1601, 1604, 1622, 1637, 1652, 1653, 1659, 1660, 1704, 1708, 1713, 1720, 1734, 1758, 1766, 1792, 1845, 1873, 1905, 1920, 2043, 2052, 2055, 2059, 2070, 2078, 2108, 2113, 2128, 2130, 2136, 2138, 2191, 2220, 2233, 2267, 2268, 2278, 2293, 2295, 2319, 2325, 2326, 2337, 2371, 2379, 2425, 2433, 2471, 2474, 2475, 2495, 2497, 2522, 2536, 2537, 2584, 2585, 2586, 2606, 2615, 2667, 2724, 2739, 2742, 1, 2, 6, 10, 13, 15, 22, 24, 31, 32, 33, 38, 41, 43, 44, 46, 48, 56, 61, 62, 67, 68, 69, 70, 72, 73, 75, 78, 80, 81, 82, 85, 88, 89, 90, 94, 102, 104, 107, 110, 112, 116, 119,

[['l4', 'c8'], ['p26', 'c1'], ['c8', 'l2'], ['p12', 'i147']] 1536475200000
[60, 81, 96, 121, 145, 146, 162, 226, 278, 307, 320, 325, 333, 341, 354, 380, 381, 415, 422, 444, 450, 451, 482, 523, 536, 603, 604, 605, 606, 609, 641, 677, 678, 679, 681, 683, 684, 692, 701, 772, 775, 785, 802, 829, 831, 833, 846, 850, 853, 855, 860, 912, 915, 916, 918, 936, 937, 965, 970, 989, 990, 1030, 1070, 1075, 1153, 1176, 1180, 1197, 1213, 1226, 1270, 1276, 1281, 1291, 1355, 1424, 1434, 1482, 1506, 1529, 1542, 1590, 1600, 1604, 1611, 1651, 1652, 1810, 1819, 1843, 1845, 1851, 1881, 1913, 1917, 1921, 2000, 2022, 2063, 2064, 2094, 2139, 2148, 2149, 2167, 2224, 2225, 2231, 2245, 2282, 2306, 2314, 2315, 2363, 2394, 2403, 2451, 2452, 2463, 2464, 2475, 2525, 2568, 2584, 2590, 2670, 2715, 2723, 2746, 2748, 2757, 2772, 2811, 2829, 2830, 2843, 2858, 2925, 2, 3, 4, 5, 7, 9, 11, 12, 13, 14, 17, 19, 20, 21, 23, 25, 26, 28, 31, 37, 39, 41, 43, 55, 58, 60, 64, 67, 76, 78, 82, 83, 84, 88, 89, 90, 91, 92, 94, 96, 97, 98

...

[['p32', 'l2', 'e12', 'c6'], ['e01', 'p08', 'c1', 'l4'], ['c7', 'e09', 'p32', 'l3'], ['c4', 'l4', 'p29', 'e05']] 1536823200000
[1, 5, 10, 13, 15, 16, 21, 28, 30, 34, 43, 44, 48, 53, 71, 72, 76, 79, 83, 86, 91, 93, 97, 98, 103, 113, 123, 126, 130, 133, 146, 147, 154, 158, 165, 168, 170, 171, 184, 187, 215, 223, 225, 231, 234, 236, 237, 242, 248, 253, 257, 259, 264, 268, 270, 271, 274, 286, 305, 306, 309, 310, 311, 319, 322, 330, 333, 336, 341, 345, 347, 349, 351, 358, 360, 367, 368, 369, 370, 378, 380, 385, 386, 395, 414, 420, 425, 433, 434, 435, 442, 444, 449, 450, 452, 453, 454, 456, 459, 460, 468, 469, 471, 481, 488, 489, 507, 508, 521, 523, 524, 531, 533, 537, 541, 553, 563, 565, 574, 579, 581, 593, 594, 595, 596, 604, 609, 610, 612, 616, 620, 633, 635, 639, 640, 641, 648, 649, 652, 653, 667, 673, 680, 682, 683, 684, 691, 694, 709, 717, 733, 734, 751, 758, 759, 764, 765, 775, 776, 777, 779, 785, 794, 801, 815, 819, 820, 822, 833, 838, 840, 841, 846, 856, 858, 859, 865, 869, 875, 880

[['p32', 'l2', 'e12', 'c6'], ['e01', 'p08', 'c1', 'l4'], ['c7', 'e09', 'p32', 'l3'], ['c4', 'l4', 'p29', 'e05'], ['e08', 'l3', 'p34', 'i11']] 1536656100000
[7, 10, 11, 12, 22, 23, 24, 28, 38, 47, 49, 69, 74, 75, 81, 85, 86, 87, 94, 98, 103, 111, 125, 128, 139, 140, 142, 144, 146, 151, 153, 157, 164, 166, 174, 175, 188, 191, 193, 194, 196, 204, 207, 219, 223, 225, 229, 232, 235, 249, 252, 256, 272, 274, 279, 287, 294, 300, 304, 306, 312, 315, 319, 331, 336, 337, 338, 341, 352, 353, 356, 357, 359, 360, 369, 370, 373, 383, 388, 394, 400, 416, 417, 418, 419, 421, 425, 435, 458, 472, 478, 484, 500, 510, 516, 517, 519, 521, 531, 546, 564, 567, 571, 572, 577, 595, 602, 606, 611, 615, 616, 618, 621, 624, 626, 631, 632, 644, 645, 646, 648, 649, 659, 661, 662, 665, 669, 685, 691, 695, 696, 697, 704, 716, 726, 728, 736, 737, 740, 750, 765, 772, 774, 775, 781, 794, 795, 826, 830, 846, 849, 857, 862, 863, 866, 870, 876, 885, 891, 909, 910, 911, 921, 926, 927, 929, 936, 945, 951, 955, 957, 960, 964,

[['c4', 'l4', 'p23', 'e05', 'i34']] 1536805800000
[0, 7, 9, 15, 17, 23, 25, 26, 28, 36, 38, 46, 49, 58, 69, 72, 76, 82, 83, 89, 93, 95, 96, 98, 108, 109, 111, 116, 120, 121, 124, 125, 128, 138, 141, 144, 153, 157, 164, 165, 170, 171, 174, 181, 189, 193, 200, 204, 205, 206, 212, 216, 217, 236, 241, 252, 253, 255, 256, 259, 263, 283, 293, 296, 308, 318, 319, 322, 327, 330, 335, 337, 343, 357, 359, 360, 366, 368, 370, 374, 376, 378, 380, 383, 387, 389, 398, 407, 411, 415, 419, 420, 423, 426, 451, 455, 456, 462, 465, 482, 485, 490, 491, 494, 497, 509, 510, 519, 523, 526, 527, 530, 538, 540, 541, 543, 545, 546, 549, 567, 568, 577, 579, 580, 582, 601, 602, 607, 608, 614, 618, 625, 629, 640, 649, 658, 659, 661, 664, 670, 673, 674, 675, 677, 679, 680, 686, 693, 699, 702, 717, 718, 720, 723, 725, 727, 740, 742, 750, 760, 762, 766, 770, 771, 772, 781, 784, 785, 786, 790, 800, 801, 807, 810, 811, 814, 816, 818, 821, 822, 832, 833, 837, 850, 852, 864, 869, 874, 876, 878, 889, 892, 896, 904, 907, 9

[['c4', 'l4', 'p23', 'e05', 'i34'], ['l1', 'p09', 'i36', 'e01', 'c8'], ['i86', 'c3', 'p05', 'l2', 'e04'], ['p26', 'e04', 'l3', 'i28', 'c2']] 1535807100000
[2, 18, 25, 28, 35, 36, 37, 40, 50, 51, 54, 56, 62, 64, 65, 66, 68, 72, 75, 80, 94, 96, 100, 103, 112, 116, 119, 129, 132, 133, 135, 138, 141, 146, 148, 149, 150, 151, 153, 155, 159, 162, 163, 164, 165, 169, 179, 180, 185, 186, 187, 190, 192, 196, 198, 202, 203, 205, 206, 210, 217, 218, 225, 228, 230, 239, 243, 244, 247, 249, 251, 261, 263, 265, 266, 268, 273, 286, 289, 292, 293, 300, 303, 312, 316, 319, 325, 328, 331, 347, 348, 351, 354, 356, 357, 358, 362, 366, 367, 369, 370, 371, 373, 374, 379, 382, 401, 403, 405, 407, 410, 411, 422, 428, 435, 436, 438, 440, 444, 448, 450, 469, 474, 475, 477, 486, 488, 493, 497, 501, 504, 505, 510, 517, 519, 520, 526, 543, 546, 547, 548, 549, 550, 561, 564, 566, 567, 572, 574, 576, 578, 583, 585, 586, 589, 597, 600, 603, 618, 623, 627, 628, 637, 638, 639, 641, 644, 652, 655, 660, 665, 667, 669, 67

[['c4', 'l4', 'p23', 'e05', 'i34'], ['l1', 'p09', 'i36', 'e01', 'c8'], ['i86', 'c3', 'p05', 'l2', 'e04'], ['p26', 'e04', 'l3', 'i28', 'c2'], ['c5', 'p07', 'e06', 'l2', 'i25']] 1536194100000
[0, 1, 5, 10, 15, 17, 21, 24, 28, 29, 31, 35, 36, 37, 40, 51, 63, 64, 68, 72, 74, 76, 78, 79, 82, 94, 96, 98, 106, 109, 110, 131, 135, 138, 139, 141, 147, 149, 150, 158, 159, 161, 164, 170, 180, 187, 195, 206, 209, 210, 214, 219, 220, 232, 233, 234, 237, 249, 251, 256, 266, 269, 273, 275, 295, 296, 300, 304, 305, 308, 313, 314, 321, 326, 339, 340, 344, 350, 356, 358, 363, 369, 373, 378, 384, 391, 402, 403, 404, 420, 428, 429, 436, 438, 441, 444, 446, 449, 450, 462, 477, 481, 487, 488, 489, 503, 519, 521, 527, 528, 529, 532, 535, 550, 551, 552, 557, 561, 562, 565, 566, 569, 572, 576, 578, 581, 583, 591, 600, 615, 618, 626, 629, 632, 635, 636, 653, 655, 666, 670, 676, 677, 681, 683, 697, 699, 705, 706, 708, 709, 713, 716, 717, 721, 724, 725, 732, 734, 736, 737, 742, 752, 757, 759, 765, 771, 773, 777, 